<a href="https://colab.research.google.com/github/PaoloComensoli/MasterDegree/blob/main/Decision%20Models/Assignments/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 6.5 MB/s 


In [10]:
from pulp import *
import pandas as pd

In [3]:
model = LpProblem("MaximizeProfit", LpMaximize)

In [4]:
# Variables
cargos = ['Cargo 1', 'Cargo 2', 'Cargo 3']
wagons = ['Wagon 1','Wagon 2','Wagon 3','Wagon 4']
key = [(c,w) for c in cargos for w in wagons]
transport = LpVariable.dicts('TRANSPORT', key, lowBound=0, cat='Continuous')

# Variables for objective functions
revenues = {
    'Cargo 1' : 3500,
    'Cargo 2' : 2500,
    'Cargo 3' : 2000
}

# Variables for constraints
cargo_availability = {
    'Cargo 1' : 20,
    'Cargo 2' : 10,
    'Cargo 3' : 18
}

cargo_space = {
    'Cargo 1' : 500,
    'Cargo 2' : 300,
    'Cargo 3' : 400
}

wagons_availability = {
    'Wagon 1' : (10, 5000),
    'Wagon 2' : (8, 4000),
    'Wagon 3' : (12, 8000),
    'Wagon 4' : (6, 2500),
}

In [5]:
# Define Objective
model += lpSum([revenues[k[0]] * transport[k] for k in key])

In [6]:
# Define Constraints
for c in cargos:
    model += lpSum([transport[k] for k in key if k[0] == c]) <= cargo_availability[c]
    
for w in wagons:
    model += lpSum([transport[k] for k in key if k[1] == w]) <= wagons_availability[w][0]

for w in wagons:
    model += lpSum([cargo_space[k[0]] * transport[k] for k in key if w[1] == w]) <= wagons_availability[w][1]

In [7]:
model.writeLP("assignment1.lp") # Store the problem
model.solve(PULP_CBC_CMD(msg=False)) # Keep the output clean
status = LpStatus[model.status]
print(status)

Optimal


In [8]:
key_values = {v: k for k, v in transport.items()}
print('Optimal value:', model.objective.value(), 'of revenue')
print('')
print('Output variables: ')

current_cargo = ''
for v in model.variables():
    cargo = key_values[v][0]
    if cargo != current_cargo:
        print('******* ' + cargo + ' *******')
        current_cargo = cargo
    print(v.varValue, 'tons of', cargo, 'on', key_values[v][1])

Optimal value: 107000.0 of revenue

Output variables: 
******* Cargo 1 *******
6.0 tons of Cargo 1 on Wagon 1
8.0 tons of Cargo 1 on Wagon 2
0.0 tons of Cargo 1 on Wagon 3
6.0 tons of Cargo 1 on Wagon 4
******* Cargo 2 *******
0.0 tons of Cargo 2 on Wagon 1
0.0 tons of Cargo 2 on Wagon 2
10.0 tons of Cargo 2 on Wagon 3
0.0 tons of Cargo 2 on Wagon 4
******* Cargo 3 *******
4.0 tons of Cargo 3 on Wagon 1
0.0 tons of Cargo 3 on Wagon 2
2.0 tons of Cargo 3 on Wagon 3
0.0 tons of Cargo 3 on Wagon 4


In [11]:
# Sensitivity - Shadow Price & Slack <- only for the old version
s = [{'Contraint Name': name, 'Shadow Price': c.pi, 'Slack': c.slack}
     for name, c in model.constraints.items()]
print(pd.DataFrame(s))

   Contraint Name  Shadow Price   Slack
0             _C1        1500.0    -0.0
1             _C2         500.0    -0.0
2             _C3          -0.0    12.0
3             _C4        2000.0    -0.0
4             _C5        2000.0    -0.0
5             _C6        2000.0    -0.0
6             _C7        2000.0    -0.0
7             _C8          -0.0  5000.0
8             _C9          -0.0  4000.0
9            _C10          -0.0  8000.0
10           _C11          -0.0  2500.0


# Sensitivity Analysis - GLPK
Prerequisites: The solver GLPK must be installed

In [ ]:
model.solve(GLPK(msg=True, options=['--ranges', 'assignment1_Sensitivity.sen'])) # Solve and produce the sensitivity report